In [ ]:
!pip install bertopic

In [28]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy

In [29]:
df_Oneplus = pd.read_csv('iPhone_reviews.csv')

In [30]:
df_Oneplus.shape

(300, 6)

In [31]:
df_Oneplus.head(10)

,Unnamed: 0.1,Unnamed: 0,rating,title,review_text,model
0,0.0,0,5.0 out of 5 stars,5.0 out of 5 stars\nThe best iPhone - perfect ...,\nThis is my first iPhone. After being an Andr...,Apple iPhone 11
1,1.0,1,5.0 out of 5 stars,5.0 out of 5 stars\nComing from Androids plann...,"\nBeen a few weeks since I got the phone, comi...",Apple iPhone 11
2,2.0,2,4.0 out of 5 stars,4.0 out of 5 stars\niPhone 11 review\n,\niPhone11 Review A person who uses every majo...,Apple iPhone 11
3,3.0,3,5.0 out of 5 stars,5.0 out of 5 stars\nNICE PRODUCT\n,\nBEST SELLER SENDING PERFECT PRODUCT\n,Apple iPhone 11
4,4.0,4,5.0 out of 5 stars,5.0 out of 5 stars\nPerfect switch from OnePlu...,\nAfter 5 years of loyalty towards OnePlus an...,Apple iPhone 11
5,5.0,5,5.0 out of 5 stars,5.0 out of 5 stars\nPerfect phone\n,\nIf u read the review you will know that ther...,Apple iPhone 11
6,6.0,6,5.0 out of 5 stars,5.0 out of 5 stars\nA detailed review\n,"\nAfter 3 months of usage, It feels smooth and...",Apple iPhone 11
7,7.0,7,5.0 out of 5 stars,5.0 out of 5 stars\nAwesome\n,\nGood quality\n,Apple iPhone 11
8,8.0,8,5.0 out of 5 stars,5.0 out of 5 stars\nClassic\n,\nWriting review after 2+years.no issues till ...,Apple iPhone 11
9,9.0,9,4.0 out of 5 stars,4.0 out of 5 stars\nBeast with a limp\n,\nBeen almost 10 months I have been using the ...,Apple iPhone 11


In [32]:
import re

def remove_ratings(title):
    return re.sub(r'\d+\.\d+ out of \d+ stars', '', title).strip()

In [33]:
df_Oneplus['title'] = df_Oneplus['title'].apply(remove_ratings)

In [34]:
df_Oneplus['reviews'] =df_Oneplus['review_text']

In [35]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mohdzahan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/mohdzahan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U00002702-\U000027B0"
    "\U000024C2-\U0001F251"
    "]+",
    flags=re.UNICODE
)

In [38]:

df_Oneplus['reviews'] = df_Oneplus['reviews'].astype(str)

In [39]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def preprocess(text):
    text = text.replace('\n', '.')
    text = emoji_pattern.sub(r'', text)  
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()  
    text = ' '.join(word for word in text.split() if word not in stop_words)  
    return text

df_Oneplus['reviews'] = df_Oneplus['reviews'].apply(preprocess)

In [40]:
nlp = spacy.load('en_core_web_md', disable = ['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
    output = []
    for text in texts:
        doc = nlp(text)
        output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return output

tokenized_reviews = lemmatization(df_Oneplus['reviews'].tolist())

In [41]:
print(tokenized_reviews)

[['first', 'iphone', 'loyalist', 'decade', 'poor', 'pixel', 'design', 'camera', 'iphone', 'switch', 'walled', 'garden', 'edge', 'lcd', 'screen', 'low', 'resolution', 'unfounded', 'screen', 'iphone', 'clear', 'good', 'color', 'accuracy', 'punch', 'lot', 'research', 'iphone', 'pro', 'max', 'model', 'note', 'oneplus', 'side', 'side', 'iphone', 'top', 'camera', 'department', 'important', 'criterion', 'choose', 'year', 'iphone', 'substantial', 'update', 'pro', 'last', 'day', 'excellent', 'battery', 'life', 'day', 'phone', 'dual', 'camera', 'system', 'normal', 'iphone', 'perfect', 'compromise', 'pro', 'model', 'primary', 'sensor', 'wide', 'angle', 'sensor', 'telephoto', 'quality', 'pro', 'picture', 'true', 'life', 'accurate', 'color', 'great', 'hdr', 'low', 'light', 'performance', 'top', 'notch', 'bionic', 'butter', 'smooth', 'experience', 'lag', 'free', 'ram', 'model', 'note', 'testing', 'promise', 'straight', 'apple', 'underrated', 'misunderstood', 'parameter', 'comparable', 'money', 'andr

In [42]:
final_reviews_Oneplus = [' '.join(review) for review in tokenized_reviews]

In [43]:
final_reviews_Oneplus

['first iphone loyalist decade poor pixel design camera iphone switch walled garden edge lcd screen low resolution unfounded screen iphone clear good color accuracy punch lot research iphone pro max model note oneplus side side iphone top camera department important criterion choose year iphone substantial update pro last day excellent battery life day phone dual camera system normal iphone perfect compromise pro model primary sensor wide angle sensor telephoto quality pro picture true life accurate color great hdr low light performance top notch bionic butter smooth experience lag free ram model note testing promise straight apple underrated misunderstood parameter comparable money android phone late os month year year security update apple year update use financial transaction security release bug fix day critical build quality design unparalleled love design apple premium brand attention detail alignment port element camera cutout gesture poor job samsung dumb implementation note sp

In [44]:
text = df_Oneplus['reviews']

In [45]:
vectorizer_model = CountVectorizer(ngram_range = (1,2), stop_words = "english")

In [46]:
model = BERTopic(vectorizer_model = vectorizer_model,language = "english",calculate_probabilities = True,verbose = True)

: 

In [47]:
topics, probs = model.fit_transform(text)

2024-07-21 23:20:24,390 - BERTopic - Embedding - Transforming documents to embeddings.


In [ ]:
model.print_topics()